In [1]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
import torch
import matplotlib.pyplot as plt


In [52]:
# Loading the training file
file_path = '/Users/diana/Desktop/isear-train.xlsx'

# Define custom headers
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

# Display the first few rows
print(len(df['Text']))

5366


In [53]:
# Label preprocessing

label_encoding = {'anger': 0, 'disgust': 1, 'fear': 2, 'guilt': 3, 'joy': 4, 'sadness': 5, 'shame': 6}
y = df['Emotions'].values
y_train_encoded = np.array([label_encoding[label] for label in y])
y_train_tensor = torch.tensor (y_train_encoded)

#print("Encoded labels:", y, y_train_encoded[:20])
#print(len(y))


In [145]:
# Custom data preprocessing

def tokenize(text):
    translation_table = str.maketrans({c: f' {c} ' if not c.isalnum() else c for c in set(text)})  # creates translation
    # table(dictionary) with the built-in function maketrans, set(text) makes an unordered collection of unique
    # elements through set comprehension, a concise way to create sets
    tokenized_text = text.translate(translation_table)  # uses translation table to add whitespace around special
    # characters and punctuation
    return tokenized_text.strip().lower().split()  # split-tokens split on space, lower-tokens made all lowercase,
    # strip-and leading or trailing whitespaces are removed from string

import string 

text = ''.join(df['Text'].astype(str))
#print (text)

tokenized_text = []
for index, sentence in enumerate(df['Text']):
    emotion_label = df['Emotions'][index]
    sentence = tokenize(sentence)
    tokenized_text.append (sentence)

# Filtering out digits and punctuation
filtered_tokens = []

for tokens in tokenized_text:
    clean_tokens = []
    
    for token in tokens:
        if all(char in string.punctuation for char in token):
            continue  # Skip punctuation tokens
        if token.isdigit():
            continue  # Skip digit tokens
        else:
            clean_tokens.append(token)  
    
    filtered_tokens.append(clean_tokens)

tokenized_text = filtered_tokens
#print (filtered_tokens)

In [5]:
# Correct misspelled words

from spellchecker import SpellChecker
from collections import Counter

# Initialize the spell checker
spell = SpellChecker()

# Flatten the list of lists 
just_tokens = []
for sentence in filtered_tokens:
    for token in sentence:
        just_tokens.append (token)


# Count the frequency of each token
token_counts = Counter(just_tokens)

# Find tokens that appear only once 
uncommon_words = [token for token in token_counts if token_counts[token] == 1]

#print (len (uncommon_words))

# Check if the uncommen words were misspelled 
misspelled_words = spell.unknown(uncommon_words)
#print (misspelled_words)

# Create a dictionary of misspelled words and their corrections
corrections = {word: spell.correction(word) for word in misspelled_words}
#print("Misspelled words and corrections:", corrections)

In [146]:
# Apply corrections to the text

def correct_spelling(text, corrections):
    corrected_text = []
    for sentence in text: # Correct each token in the sentence
        corrected_tokens = [corrections.get(token, token) if corrections.get(token, token) else token for token in sentence]  # Correct the tokens
        corrected_sentence = ' '.join(corrected_tokens)  # Join the tokens back into a string (sentence)
        corrected_text.append(corrected_sentence)
    return corrected_text

# Apply spell correction to each sublist of sentences in the list of lists
corrected_texts = [correct_spelling([sentence], corrections) for sentence in filtered_tokens]
#print (corrected_texts)
df['Text'] = corrected_texts
#print (df['Text'])

#print (corrected_texts)
# Flatten the lists to a list with sentences
corr_strings = [item for sublist in corrected_texts for item in sublist]
#print (corr_strings)


In [7]:
# Creating RoBERTa embeddings for the trainset
# Load pre-trained RoBERTa model and tokenizer 
roberta_model = RobertaModel.from_pretrained('roberta-base', output_hidden_states=True)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
roberta_model.to(device)

# Function to get sentence embeddings for a list of text strings
def get_sentence_embeddings(text_list, batch_size=32):
    all_embeddings = []

    for i in range(0, len(text_list), batch_size): # Process the text list in batches
        batch = text_list[i:i + batch_size]

        # Tokenize the batch of text strings and convert to tensors
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        # No gradient calculation in forward pass
        with torch.no_grad():
            outputs = roberta_model(**inputs)

        # Extract the hidden state of the CLS token (first token) from the last layer
        cls_embeddings = outputs.hidden_states[-1][:, 0, :] 
        all_embeddings.append(cls_embeddings)   # Append the embeddings to the embedding list
 
    # Concatenate all batch embeddings
    sentence_embeddings = torch.cat(all_embeddings, dim=0)
    
    return sentence_embeddings

# Example usage with strings_dev
# sentence_embeddings_dev = get_sentence_embeddings(strings_dev)
# print("Sentence embeddings shape:", sentence_embeddings_dev.shape)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#Initialize embeddings
sentence_embeddings = get_sentence_embeddings(corr_strings)


In [147]:
# Load the devset
file_path_dev = '/Users/diana/Desktop/isear-validation.xlsx'
custom_headers_dev = ['Emotions_dev', 'Text_dev']
df_dev = pd.read_excel(file_path_dev, skiprows=1, header=None, names=custom_headers_dev)
#print (type(df_dev['Text_dev']))

# Data and label processing for devset
strings_dev = []
for index, sentence in enumerate(df_dev['Text_dev']):
    emotion_label_dev = df_dev['Emotions_dev'][index]
    strings_dev.append (sentence)
#print (strings_dev[0:3])

# Label encoding
y_dev = df_dev['Emotions_dev'].values
y_dev_encoded = np.array([label_encoding[label] for label in y_dev])
y_dev_tensor = torch.tensor (y_dev_encoded)
#print (len (strings_dev))
#print (y_dev_tensor.shape)


In [148]:
# Load the test set
file_path_test = '/Users/diana/Desktop/isear-test.xlsx'
custom_headers_test = ['Emotions_test', 'Text_test']
df_test = pd.read_excel(file_path_test, skiprows=1, header=None, names=custom_headers_test)

# Data and label processing for testset
strings_test = []
for index, sentence in enumerate(df_test['Text_test']):
    emotion_label_test = df_test['Emotions_test'][index]
    strings_test.append (sentence)

strings_test = df_test['Text_test'].tolist()
#print (strings_test[0:3])

# Label encoding
y_test = df_test['Emotions_test'].values
y_test_encoded = np.array([label_encoding[label] for label in y_test])
y_test_tensor = torch.tensor (y_test_encoded)
print (len (df_test['Text_test']))


1150


In [149]:
# Checking misspelled words in the testset
# Convert the sentences to a single string
text_test = ' '.join(df_test['Text_test'].astype(str))

# Tokenize the testset 
tokenized_text_test = []
for index, sentence in enumerate(df_test['Text_test']):
    emotion_label = df_test['Emotions_test'][index]
    tokenized_sentence = tokenize(sentence)
    tokenized_text_test.append(tokenized_sentence)

filtered_tokens_test = []

# Filter out punctuation and digit tokens
for tokens in tokenized_text_test:
    clean_tokens = []
    for token in tokens:
        if all(char in string.punctuation for char in token):
            continue  # Skip punctuation tokens
        if token.isdigit():
            continue  # Skip digit tokens
        else:
            clean_tokens.append(token)
    filtered_tokens_test.append(clean_tokens)

#print (filtered_tokens_test)



In [57]:
# Initialize spellchecker
spell = SpellChecker()

# Flatten the list of lists into a single list
just_tokens_test = [token for sentence in filtered_tokens_test for token in sentence]
#Count all tokens
token_counts_test = Counter(just_tokens_test)

# Find the tokens that appear only once
uncommon_words_test = [token for token in token_counts_test if token_counts_test[token] == 1]
#Check for misspellings
misspelled_words_test = spell.unknown(uncommon_words_test)

#print(f"Misspelled words: {misspelled_words_test}")
corrections_test = {word: spell.correction(word) for word in misspelled_words_test}
#print(f"Misspelled words and corrections: {corrections_test}")
#print (len(misspelled_words_test))


In [150]:
# Apply spelling corrections to each sentence in the text
corrected_texts_test = [correct_spelling([sentence], corrections_test) for sentence in filtered_tokens_test]
#print (len (corrected_texts_test))

corr_strings_test = df_test['Text_test']
corr_strings_test = [item for sublist in corrected_texts_test for item in sublist]
#print (corr_strings_test)


In [13]:
# Creating devset embeddings
sentence_embeddings_dev = get_sentence_embeddings(strings_dev)
# Print the shape of the embeddings
print("Sentence embeddings shape:", sentence_embeddings_dev.shape)

Sentence embeddings shape: torch.Size([1150, 768])


In [82]:
#Creating testset embeddings
sentence_embeddings_test = get_sentence_embeddings(corr_strings_test)
#print("Sentence embeddings shape:", sentence_embeddings_test.shape)

In [15]:
# Checking dimensions
#print (type (sentence_embeddings))
y_train_tensor = torch.tensor(y_train_encoded)
y_test_tensor = torch.tensor(y_test_encoded)
print (y_dev_tensor.shape, y_test_tensor.shape,y_train_tensor.shape) 

torch.Size([1150]) torch.Size([1150]) torch.Size([5366])


In [98]:
# Adding a disgust embedding feature
import torch
from nltk.tokenize import word_tokenize

corr_strings = pd.Series(corr_strings)
corr_strings_test = pd.Series(corr_strings_test)

# List of unique disgust words identified earlier
unique_disgust_words = ['prostitute', 'detested', 'disgusting', 'disgusted', 'disgust', 'vomit', 'rotten', 'stink', 'spider', 'contempt', 'drunk', 'revolt']

# Function to check if any of the disgust words are in the text

def disgust_words(corrtext, disgust_words):
    tokens = word_tokenize(corrtext.lower())
    """if any(word in tokens for word in disgust_words):
        #print(f"yes: {tokens}")
        matches = sum(1 for word in tokens if word in disgust_words)
        #print (matches)"""
    return int(any(word in tokens for word in disgust_words))


# Apply the function to create the disgust feature
df['disgust_feature_train'] = (corr_strings).apply(lambda x: disgust_words(x, unique_disgust_words))
df_test['disgust_feature_test'] = (corr_strings_test).apply(lambda x: disgust_words(x, unique_disgust_words))
df_dev['disgust_feature_dev'] = (df_dev['Text_dev']).apply(lambda x: disgust_words(x, unique_disgust_words))


# Extract the binary disgust feature as a numpy array and convert to tensor
disgust_features_train = torch.tensor(df['disgust_feature_train'].values, dtype=torch.float32).reshape(-1, 1)
disgust_features_test = torch.tensor(df_test['disgust_feature_test'].values, dtype=torch.float32).reshape(-1, 1)
disgust_features_dev = torch.tensor(df_dev['disgust_feature_dev'].values, dtype=torch.float32).reshape(-1, 1)

# Assuming sentence_embeddings, sentence_embeddings_test, and sentence_embeddings_dev are already defined
embeddings_train = torch.tensor(sentence_embeddings, dtype=torch.float32).to(device)
embeddings_test = torch.tensor(sentence_embeddings_test, dtype=torch.float32).to(device)
embeddings_dev = torch.tensor(sentence_embeddings_dev, dtype=torch.float32).to(device)

# Ensure the lengths match before concatenation
assert embeddings_train.shape[0] == disgust_features_train.shape[0], "Train set sizes do not match"
assert embeddings_test.shape[0] == disgust_features_test.shape[0], "Test set sizes do not match"
assert embeddings_dev.shape[0] == disgust_features_dev.shape[0], "Dev set sizes do not match"

# Concatenate the RoBERTa embeddings with the disgust feature
augmented_embeddings_train = torch.cat((embeddings_train, disgust_features_train), dim=1)
augmented_embeddings_test = torch.cat((embeddings_test, disgust_features_test), dim=1)
augmented_embeddings_dev = torch.cat((embeddings_dev, disgust_features_dev), dim=1)

#print(augmented_embeddings_train.shape[1])"""

/var/folders/mh/2g8pjxqx2dldlw8dltkfmgvm0000gn/T/ipykernel_954/574596504.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_train = torch.tensor(sentence_embeddings, dtype=torch.float32).to(device)
/var/folders/mh/2g8pjxqx2dldlw8dltkfmgvm0000gn/T/ipykernel_954/574596504.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_test = torch.tensor(sentence_embeddings_test, dtype=torch.float32).to(device)
/var/folders/mh/2g8pjxqx2dldlw8dltkfmgvm0000gn/T/ipykernel_954/574596504.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTen

In [99]:
# Adding an anger embedding feature
unique_anger_words = ['anger', 'angry', 'rage', 'wrath', 'furious', 'enraged', 'irritated', 'kick', 'abuse']

# Function to check if any of the anger words are in the text
def anger_words(corrtext, anger_words):
    tokens = word_tokenize(corrtext.lower())
    """if any(word in tokens for word in anger_words):
        #print(f"yes: {tokens}")"""
    return int(any(word in tokens for word in anger_words))

# Apply the function to create the anger feature
df['anger_feature_train'] = (corr_strings).apply(lambda x: anger_words(x, unique_anger_words))
df_test['anger_feature_test'] = (corr_strings_test).apply(lambda x: anger_words(x, unique_anger_words))
df_dev['anger_feature_dev'] = df_dev['Text_dev'].apply(lambda x: anger_words(x, unique_anger_words))

# Extract the binary anger feature as a numpy array and convert to tensor
anger_features_train = torch.tensor(df['anger_feature_train'].values, dtype=torch.float32).reshape(-1, 1)
anger_features_test = torch.tensor(df_test['anger_feature_test'].values, dtype=torch.float32).reshape(-1, 1)
anger_features_dev = torch.tensor(df_dev['anger_feature_dev'].values, dtype=torch.float32).reshape(-1, 1)

# Ensure the lengths match before concatenation
assert embeddings_train.shape[0] == anger_features_train.shape[0], "Train set sizes do not match"
assert embeddings_test.shape[0] == anger_features_test.shape[0], "Test set sizes do not match"
assert embeddings_dev.shape[0] == anger_features_dev.shape[0], "Dev set sizes do not match"

# Concatenate the RoBERTa embeddings with the anger feature
augmented_embeddings_train_a = torch.cat((augmented_embeddings_train, anger_features_train), dim=1)
augmented_embeddings_test_a = torch.cat((augmented_embeddings_test, anger_features_test), dim=1)
augmented_embeddings_dev_a = torch.cat((augmented_embeddings_dev, anger_features_dev), dim=1)

print(augmented_embeddings_train_a.shape[1])

770


In [100]:
# Adding a shame embedding feature
unique_shame_words = ['ashamed', 'embarrassed', 'humiliated', 'shame', 'regret', 'disgrace', 'dishonor', 'reject']

# Function to check if any of the shame words are in the text
def shame_words(corrtext, shame_words):
    tokens = word_tokenize(corrtext.lower())
    """if any(word in tokens for word in shame_words):
        print(f"yes")"""
    return int(any(word in tokens for word in shame_words))

# Apply the function to create the shame feature
df['shame_feature_train'] = (corr_strings).apply(lambda x: shame_words(x, unique_shame_words))
df_test['shame_feature_test'] = (corr_strings_test).apply(lambda x: shame_words(x, unique_shame_words))
df_dev['shame_feature_dev'] = df_dev['Text_dev'].apply(lambda x: shame_words(x, unique_shame_words))

# Extract the binary shame feature as a numpy array and convert to tensor
shame_features_train = torch.tensor(df['shame_feature_train'].values, dtype=torch.float32).reshape(-1, 1)
shame_features_test = torch.tensor(df_test['shame_feature_test'].values, dtype=torch.float32).reshape(-1, 1)
shame_features_dev = torch.tensor(df_dev['shame_feature_dev'].values, dtype=torch.float32).reshape(-1, 1)

# Ensure the lengths match before concatenation
assert embeddings_train.shape[0] == shame_features_train.shape[0], "Train set sizes do not match"
assert embeddings_test.shape[0] == shame_features_test.shape[0], "Test set sizes do not match"
assert embeddings_dev.shape[0] == shame_features_dev.shape[0], "Dev set sizes do not match"

# Concatenate the RoBERTa embeddings with the shame feature
augmented_embeddings_train_shame = torch.cat((augmented_embeddings_train_a, shame_features_train), dim=1)
augmented_embeddings_test_shame = torch.cat((augmented_embeddings_test_a, shame_features_test), dim=1)
augmented_embeddings_dev_shame = torch.cat((augmented_embeddings_dev_a, shame_features_dev), dim=1)

print(augmented_embeddings_train_shame.shape[1])

771


In [101]:
# Adding a feature for fear
#List of unique fear words identified earlier
unique_fear_words = ['afraid', 'fear', 'panic', 'scare', 'worry', 'anxious', 'frighten', 'dark']

# Function to check if any of the disgust words are in the text
def fear_words(text, fear_words):
    tokens = word_tokenize(text.lower())
    """if any(word in tokens for word in fear_words):
        #print ("yes")"""
    return int(any(word in tokens for word in fear_words))
    

# Apply the function to create the disgust feature
df['fear_feature_train'] = (corr_strings).apply(lambda x: fear_words(x, unique_fear_words))
df_test['fear_feature_test'] = (corr_strings_test).apply(lambda x: fear_words(x, unique_fear_words))
df_dev['fear_feature_dev'] = df_dev['Text_dev'].apply(lambda x: fear_words(x, unique_fear_words))

# Extract the binary disgust feature as a numpy array and convert to tensor
fear_features_train = torch.tensor(df['fear_feature_train'].values, dtype=torch.float32).reshape(-1, 1)
fear_features_test = torch.tensor(df_test['fear_feature_test'].values, dtype=torch.float32).reshape(-1, 1)
fear_features_dev = torch.tensor(df_dev['fear_feature_dev'].values, dtype=torch.float32).reshape(-1, 1)


# Ensure the lengths match before concatenation
assert embeddings_train.shape[0] == fear_features_train.shape[0], "Train set sizes do not match"
assert embeddings_test.shape[0] == fear_features_test.shape[0], "Test set sizes do not match"
assert embeddings_dev.shape[0] == fear_features_dev.shape[0], "Dev set sizes do not match"

# Concatenate the RoBERTa embeddings with the anger feature
augmented_embeddings_train_fear = torch.cat((augmented_embeddings_train_shame, fear_features_train), dim=1)
augmented_embeddings_test_fear= torch.cat((augmented_embeddings_test_shame, fear_features_test), dim=1)
augmented_embeddings_dev_fear = torch.cat((augmented_embeddings_dev_shame, fear_features_dev), dim=1)

print (augmented_embeddings_train_fear.shape[1])

772


In [119]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.init as init


# Convert to PyTorch tensors
embeddings = torch.tensor(augmented_embeddings_train_fear, dtype=torch.float32).to(device)
labels = torch.tensor(y_train_encoded, dtype=torch.long).to(device)

embeddings_dev = torch.tensor(augmented_embeddings_dev_fear, dtype=torch.float32).to(device)
labels_dev = torch.tensor(y_dev_encoded, dtype=torch.long)

embeddings_test = torch.tensor(augmented_embeddings_test_fear, dtype=torch.float32).to(device)
labels_test = torch.tensor(y_test_encoded, dtype=torch.long)

# Create datasets and dataloaders for training, validation, and testing
dataset = TensorDataset(embeddings, labels)
dataset_dev = TensorDataset(embeddings_dev,labels_dev)
dataset_test = TensorDataset(embeddings_test,labels_test)


train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset_dev, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.bn1 = nn.BatchNorm1d(hidden_dim1)
        self.relu1 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.relu2 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.bn3 = nn.BatchNorm1d(hidden_dim3)
        self.relu3 = nn.LeakyReLU()
        self.dropout3 = nn.Dropout(0.4) 

        self.fc4 = nn.Linear(hidden_dim3, output_dim)


        self.init_weights()

    def init_weights(self):
        # Initialize weights and biases using Kaiming initialization and constant zero for biases
        init.kaiming_uniform_(self.fc1.weight, nonlinearity='leaky_relu')
        init.constant_(self.fc1.bias, 0)
        init.kaiming_uniform_(self.fc2.weight, nonlinearity='leaky_relu')
        init.constant_(self.fc2.bias, 0)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity='leaky_relu')
        init.constant_(self.fc3.bias, 0)
        init.kaiming_uniform_(self.fc4.weight, nonlinearity='leaky_relu')
        init.constant_(self.fc4.bias, 0)

    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        return x
    
# Define dimensions for the neural network
input_dim = sentence_embeddings.shape[1] + 4
hidden_dim1 = 568  
hidden_dim2 = 284   
hidden_dim3 = 16
output_dim = 7

/var/folders/mh/2g8pjxqx2dldlw8dltkfmgvm0000gn/T/ipykernel_954/3014603393.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = torch.tensor(augmented_embeddings_train_fear, dtype=torch.float32).to(device)
/var/folders/mh/2g8pjxqx2dldlw8dltkfmgvm0000gn/T/ipykernel_954/3014603393.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_dev = torch.tensor(augmented_embeddings_dev_fear, dtype=torch.float32).to(device)
/var/folders/mh/2g8pjxqx2dldlw8dltkfmgvm0000gn/T/ipykernel_954/3014603393.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tens

In [120]:
# Create an instan of the model
simp_model = SimpleNN(input_dim, hidden_dim1, output_dim).to(device)
# Define Loss and Optimisation function
lossf = nn.CrossEntropyLoss()
optimizer = optim.AdamW (simp_model.parameters(), lr=0.0001, weight_decay=1e-9) # L2 regularization 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)


In [144]:
num_epochs = 30

# Initialize lists to store losses
train_losses = []
val_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    simp_model.train()
    train_loss = 0.0
    
    for batch_embeddings, batch_labels in train_loader:
        # Forward pass
        outputs = simp_model(batch_embeddings)
        loss = lossf(outputs, batch_labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # Calculate average training loss
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation phase
    simp_model.eval()
    val_loss = 0.0

    
    with torch.no_grad():
        for batch_embeddings, batch_labels in val_loader:
            outputs = simp_model(batch_embeddings)
            loss = lossf(outputs, batch_labels)
            val_loss += loss.item()
    
    # Calculate average validation loss
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    scheduler.step(val_loss)
    
    # Test phase
    simp_model.eval()
    test_loss = 0.0
    
    with torch.no_grad():
        for batch_embeddings, batch_labels in test_loader:
            outputs = simp_model(batch_embeddings)
            loss = lossf(outputs, batch_labels)
            test_loss += loss.item()
    
    # Calculate average test loss
    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Test Loss: {test_loss:.4f}')


Epoch [1/30], Train Loss: 1.1738, Val Loss: 1.5698, Test Loss: 1.0503
Epoch [2/30], Train Loss: 1.1822, Val Loss: 1.5845, Test Loss: 1.0469
Epoch [3/30], Train Loss: 1.1734, Val Loss: 1.6298, Test Loss: 1.0620
Epoch [4/30], Train Loss: 1.1831, Val Loss: 1.5784, Test Loss: 1.0442
Epoch [5/30], Train Loss: 1.1780, Val Loss: 1.6087, Test Loss: 1.0453
Epoch [6/30], Train Loss: 1.1777, Val Loss: 1.5666, Test Loss: 1.0466
Epoch [7/30], Train Loss: 1.1698, Val Loss: 1.5908, Test Loss: 1.0456
Epoch [8/30], Train Loss: 1.1692, Val Loss: 1.5755, Test Loss: 1.0491
Epoch [9/30], Train Loss: 1.1887, Val Loss: 1.5789, Test Loss: 1.0458
Epoch [10/30], Train Loss: 1.1899, Val Loss: 1.5926, Test Loss: 1.0545
Epoch [11/30], Train Loss: 1.1727, Val Loss: 1.6089, Test Loss: 1.0415
Epoch [12/30], Train Loss: 1.1734, Val Loss: 1.6077, Test Loss: 1.0439
Epoch [13/30], Train Loss: 1.1836, Val Loss: 1.5616, Test Loss: 1.0526
Epoch [14/30], Train Loss: 1.1663, Val Loss: 1.5913, Test Loss: 1.0474
Epoch [15/30], 

In [151]:
# Evaluate the model
from sklearn.metrics import f1_score

simp_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_embeddings, batch_labels in test_loader:
        outputs = simp_model(batch_embeddings)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    f1 = f1_score(predicted , batch_labels, average='weighted')
    f1_macro = f1_score(predicted, batch_labels, average='macro')
    print (f1, f1_macro)
    print(f'Accuracy of the model on the data: {100 * correct / total:.2f}%')

0.7057239057239058 0.6380127808699237
Accuracy of the model on the data: 65.74%


In [152]:
# Prediction for each class

class_correct = np.zeros(output_dim)
class_total = np.zeros(output_dim)

# Collect true and predicted labels for calculating F1 score
true_labels = []
pred_labels = []

# Calculate correct and total predictions for each class
for true_label, pred_label in zip(batch_labels, predicted):
    true_labels.append(true_label)
    pred_labels.append(pred_label)
    if true_label == pred_label:
        class_correct[true_label] += 1
    class_total[true_label] += 1

# Calculate accuracy for each class
class_accuracies = class_correct / class_total

# Calculate F1 score for each class
f1_scores = f1_score(true_labels, pred_labels, average=None)

# Printing accuracy and F1 score for each class
for label, (accuracy, f1) in enumerate(zip(class_accuracies, f1_scores)):
    print(f"Accuracy for class {label}: {accuracy:.4f}, F1 score: {f1:.4f}")

Accuracy for class 0: 0.4000, F1 score: 0.5714
Accuracy for class 1: 1.0000, F1 score: 0.5000
Accuracy for class 2: 0.3333, F1 score: 0.5000
Accuracy for class 3: 0.8889, F1 score: 0.8889
Accuracy for class 4: 1.0000, F1 score: 0.8889
Accuracy for class 5: 0.5000, F1 score: 0.5455
Accuracy for class 6: 1.0000, F1 score: 0.5714
